In [1]:
import numpy as np
import pandas as pd
import pandasql as ps
from pandas import Series, DataFrame
import datetime as dt
import matplotlib.pyplot as plt
import sys
import time

### Отчет по странам с небольшим или отрицательным профитом

In [2]:
import pyodbc 
#Connect to OCSDBREP1 (BSS)
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.18.11.82;DATABASE=10028;UID=BSS;PWD=iKQVm40AZAmyRaw72LeY')

In [3]:
sql_srt=\
'''
--Calculate total charge and cost by MCC, MNC, usage_type, account_id
select CONVERT(VARCHAR(8), u.usage_date, 3) as 'REP_DATE'
,u.mcc,u.mnc,u.usage_type,uc.account_id
,sum(u.quantity) as quantity,sum(uc.charge) as charge,sum(uc.cost) as cost from [10028_usage2018-04].[dbo].[USAGE] u
inner join [10028_usage2018-04].[dbo].[USAGE_CHARGES] uc
on u.USAGE_ID = uc.USAGE_ID
group by CONVERT(VARCHAR(8), u.usage_date, 3),u.mcc,u.mnc,u.usage_type,uc.account_id
order by u.mcc,CONVERT(VARCHAR(8), u.usage_date, 3)
'''

In [4]:
df = pd.read_sql_query(sql_srt, cnxn,parse_dates={'REP_DATE': '%d/%m/%y'})
df.head()

,REP_DATE,mcc,mnc,usage_type,account_id,quantity,charge,cost
0,2018-04-01,202,5,15,352321.0,4.05000,0.00000,0.00000
1,2018-04-01,202,5,15,352370.0,278.61663,0.00000,0.00000
2,2018-04-01,202,5,21,352321.0,18.00000,0.00000,0.00000
3,2018-04-01,202,5,21,352334.0,26.00000,0.00000,0.00000
4,2018-04-01,202,5,22,352334.0,63.00000,0.63945,0.37044


In [5]:
def f(x):
    data=x[['charge','cost']]
    x['profit']=data['charge']-data['cost']
    if data['cost']!=0:
        x['profit_per']=round((x['profit']/data['cost'])*100,1)
    else:
        x['profit_per']=0
    return x

df1=df.groupby(['mcc','mnc']).agg({'charge': np.sum, 'cost': np.sum}).apply(f,axis=1).copy()

In [6]:
df1[df1['profit']<0]

charge         cost      profit  profit_per
mcc mnc                                                 
208 10      2.37834     34.06715   -31.68881       -93.0
    29    233.39442    274.69781   -41.30339       -15.0
214 9      24.09512    676.18343  -652.08831       -96.4
234 3       0.27545      0.91020    -0.63475       -69.7
240 10      0.07635      0.14915    -0.07280       -48.8
310 260    10.61740    677.87273  -667.25533       -98.4
    680  1465.88595   2125.99344  -660.10749       -31.0
515 2    6012.19448  10894.22103 -4882.02655       -44.8
901 18      0.74520      8.12001    -7.37481       -90.8

In [7]:
df2=df.groupby(['REP_DATE','mcc','mnc']).agg({'charge': np.sum, 'cost': np.sum}).apply(f,axis=1).copy()
df2.head()

,,,charge,cost,profit,profit_per
REP_DATE,mcc,mnc,,,,
2018-02-16,530,1,0.00026,0.00015,0.00011,73.3
2018-02-21,222,10,0.78593,0.45280,0.33313,73.6
2018-02-22,222,10,0.28601,0.16414,0.12187,74.2
2018-02-23,222,10,0.05382,0.02994,0.02388,79.8
2018-02-24,222,10,0.09339,0.05312,0.04027,75.8


In [9]:
df2[(df2.index.get_level_values('mcc')==310) & (df2.index.get_level_values('mnc')==680)].head()

,,,charge,cost,profit,profit_per
REP_DATE,mcc,mnc,,,,
2018-04-01,310,680,78.21434,72.55151,5.66283,7.8
2018-04-02,310,680,42.00338,53.25511,-11.25173,-21.1
2018-04-03,310,680,64.79633,61.03443,3.76190,6.2
2018-04-04,310,680,173.01345,144.49367,28.51978,19.7
2018-04-05,310,680,29.24078,47.55932,-18.31854,-38.5


### Анализ роста Charge
На примере Telzar (Telzar_TAP.csv)
Подходит для анализа роста на данных с небольшой девиацией.

Анализ выполняется с помощью поиска min, max, std значений

In [ ]:
import pyodbc 
#Connect to OCSDBREP1 (BSS)
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.18.11.82;DATABASE=10028;UID=BSS;PWD=iKQVm40AZAmyRaw72LeY')

sql_srt=\
'''
select CONVERT(VARCHAR(8), u.usage_date, 3) as 'REP_DATE'
,u.mcc,uc.account_id
--,u.usage_type
,sum(u.quantity) as quantity,sum(uc.charge) as charge,sum(uc.cost) as cost
from [10028_usage2018-05].[dbo].[USAGE] u
inner join [10028_usage2018-05].[dbo].[USAGE_CHARGES] uc
on u.USAGE_ID = uc.USAGE_ID
where u.usage_date >= '2018-05-04 00:00:00.000' and u.usage_date < '2018-05-15 00:00:00.000'
group by CONVERT(VARCHAR(8), u.usage_date, 3),u.mcc,uc.account_id--,u.usage_type
order by u.mcc,CONVERT(VARCHAR(8), u.usage_date, 3)
'''

df = pd.read_sql_query(sql_srt, cnxn, parse_dates={'REP_DATE': '%d/%m/%y'})
df.head()

In [15]:
df=pd.read_csv('Telzar_TAP.csv',sep=";",names=[
'USAGE_ID',
'USAGE_DATE',
'USAGE_TYPE',
'SUBSCRIBER_ID',
'DEST_PHONE_NUMBER',
'FILE_ID',
'STATUS',
'MCC',
'MNC',
'RATE',
'PLMN_CODE',
'IMEI',
'SGSN',
'DMI_MNC',
'DMI_MCC',
'GUIDING_INFO',
'SUBSCRIBER_PLAN_ID',
'SUBSCRIBER_PLAN_RULE_ID',
'QUANTITY',
'CAMEL_DESTINATION',
'CALL_NOTE',
'DIRECTION',
'SUBSCRIBER_PLAN_SERVICE_ID',
'SESSION_ID',
'USAGE_CHARGE_ID',
'USAGE_ID_1',
'PLAN_ID',
'RULE_ID',
'ACCOUNT_ID',
'CHARGE',
'COST',
'BALANCE',
'CHARGE_DATE',
'voip_charge',
'VOIP_COST',
'CHARGE_CURRENCY_ID',
'CHARGE_RATE',
'CHARGE_USD_CONVERT_RATE',
'COST_PLAN_ID',
'COST_RULE_ID',
'COST_CURRENCY_ID',
'COST_RATE',
'COST_USD_CONVERT_RATE',
'IS_SUBSCRIBER_CHARGE'
]
,usecols=[
'USAGE_DATE',
'USAGE_TYPE',
'MCC',
'MNC',
'PLMN_CODE',
'QUANTITY',
'CHARGE',
'COST'
]
#,index_col=0
,parse_dates=['USAGE_DATE'])
df.head()

,USAGE_DATE,USAGE_TYPE,MCC,MNC,PLMN_CODE,QUANTITY,CHARGE,COST
0,2018-04-01 00:00:03,15,255,3,UKRKS,0.63333,0.00000,0.00000
1,2018-04-01 00:00:05,33,334,3,MEXMS,7.25623,0.27537,0.15642
2,2018-04-01 00:00:11,33,460,0,CHNCT,0.04263,0.00072,0.00032
3,2018-04-01 00:00:16,33,219,2,HRVT2,0.02940,0.00018,0.00007
4,2018-04-01 00:00:16,33,222,10,ITAOM,0.02930,0.00021,0.00000


In [16]:
df['USAGE_DATE']=df['USAGE_DATE'].dt.floor('d')
df.head()

,USAGE_DATE,USAGE_TYPE,MCC,MNC,PLMN_CODE,QUANTITY,CHARGE,COST
0,2018-04-01,15,255,3,UKRKS,0.63333,0.00000,0.00000
1,2018-04-01,33,334,3,MEXMS,7.25623,0.27537,0.15642
2,2018-04-01,33,460,0,CHNCT,0.04263,0.00072,0.00032
3,2018-04-01,33,219,2,HRVT2,0.02940,0.00018,0.00007
4,2018-04-01,33,222,10,ITAOM,0.02930,0.00021,0.00000


In [17]:
df1=df.groupby(['USAGE_DATE','MCC']).agg({'CHARGE':np.sum}).copy() #.sort_index(level=['MCC','USAGE_DATE'])
df1.sort_index(level=['MCC','USAGE_DATE']).head(5)

,,CHARGE
USAGE_DATE,MCC,
2018-04-01,202,64.73094
2018-04-02,202,82.04028
2018-04-03,202,166.90755
2018-04-04,202,265.61592
2018-04-05,202,80.68827


In [18]:
def f(df_x):
    df_x['std_per']=round(df_x['CHARGE','std']*100/df_x['CHARGE','mean'],1)
    return df_x

df2=df1.groupby(level=['MCC']).agg({'CHARGE':[np.sum,'min','mean','max','std']}).copy()
df2.apply(f,axis=1).sort_values(by=[('CHARGE','sum'),'std_per'],ascending=False).head()

CHARGE                                                  std_per
             sum        min         mean         max          std        
MCC                                                                      
416  12259.13455   66.93419  2451.826910  6906.74036  2848.694674   116.2
460   3459.51158  460.56812   691.902316   837.36371   144.352992    20.9
440   2602.60826  235.22651   520.521652  1160.73521   383.067809    73.6
286   1448.61306   52.42763   289.722612   474.54231   173.081549    59.7
520   1211.24406  116.41286   242.248812   527.10023   170.489366    70.4